$\newcommand{\is}{\mathrel{\mathop:=}}$
$\newcommand{\range}{\mathop{ran}}$
$\newcommand{\setof}[1]{\left \{ #1 \right \}}$
$\newcommand{\card}[1]{\left | #1 \right |}$
$\newcommand{\tuple}[1]{\left \langle #1 \right \rangle}$
$\newcommand{\emptytuple}{\left \langle \right \rangle}$
$\newcommand{\tuplecat}{\cdot}$
$\newcommand{\stringcat}{\cdot}$
$\newcommand{\emptystring}{\varepsilon}$
$\newcommand{\String}[1]{\mathit{#1}}$
$\newcommand{\LeftEdgeSymbol}{\rtimes}$
$\newcommand{\RightEdgeSymbol}{\ltimes}$
$\newcommand{\LeftEdge}{\LeftEdgeSymbol}$
$\newcommand{\RightEdge}{\RightEdgeSymbol}$
$\newcommand{\mult}{\times}$
$\newcommand{\multisum}{\uplus}$
$\newcommand{\multimult}{\otimes}$
$\newcommand{\freqsymbol}{\mathrm{freq}}$
$\newcommand{\freq}[1]{\freqsymbol(#1)}$
$\newcommand{\prob}{P}$
$\newcommand{\count}[2]{\card{#2}_{#1}}$
$\newcommand{\inv}[1]{#1^{-1}}$
$\newcommand{\Lex}{\mathit{Lex}}$
$\newcommand{\length}[1]{\left | #1 \right |}$
$\newcommand{\succ}{S}$
$\newcommand{\sprec}{<}$
$\newcommand{\Rcomp}[2]{#1 \circ #2}$
$\newcommand{\domsymbol}{\triangleleft}$
$\newcommand{\idom}{\domsymbol}$
$\newcommand{\pdom}{\domsymbol^+}$
$\newcommand{\rdom}{\domsymbol^*}$
$\newcommand{\indegree}[1]{\mathrm{in(#1)}}$
$\newcommand{\outdegree}[1]{\mathrm{out(#1)}}$
$\newcommand{\cupdot}{\cup\mkern-11.5mu\cdot\mkern5mu}$
$\newcommand{\pmatrix}[1]{\left ( \matrix{#1} \right )}$

# Grammars with Relations over Strings

We have previously encountered two types of local grammars.
A strictly $k$-local (SL-$k$) grammar extracts all $k$-grams from a string and compares that against the $k$-grams listed in the grammar.
A tier-based strictly $k$-local (TSL-$k$) grammar also has a fixed set $S$ of symbols that are all removed from the string before $k$-grams are extracted.
But in both cases the grammars look at contiguous subparts of a specific string when they extract the $k$-grams.
In other words, they are extracting *substrings*.
With relations, one can instead look at *subsequences*, which also include non-contiguous parts of a string.
This yields a new type of grammar called *strictly $k$-piecewise*.

## Substrings

Any contiguous part of a string is called a *substring*.
This includes $\emptystring$, because every string $s$ can be taken to contain the empty string at any arbitrary point.
For example, *John likes Mary* is the result of $\tuple{\text{John likes Mary}} \tuplecat \emptystring$, so $\emptystring$ is a part of this string.
For the very same reason, we also regard each string as a substring of itself.

<div class=definition>
For every string $s \in \Sigma^*$, $t$ is a *substring* of $s$ iff there exist $u, v \in \Sigma*$ such that $s = u \tuplecat t \tuplecat v$.
In the special case where $u = \emptystring$, $t$ is a *prefix* of $s$, and it is a *suffix* of $s$ iff $v = \emptystring$.
</div>

<div class=example>
Consider the English word *undo*.
Choosing as our $\Sigma$ the letters of the Latin alphabet, we get the following substrings for *undo*.
$$
$$\begin{array}{rrrr}
    \text{u} & \text{n} & \text{d} & \text{o}\\
    \text{un} & \text{nd} & \text{do}\\
    \text{und} & \text{ndo}\\
    \text{undo}\\
    \emptystring\\
\end{array}$$
$$
All strings in the first column are prefixes, and all the strings along the diagonal are suffixes.
Note that $\emptystring$ and *undo* counts as both prefixes and suffixes. 
As is clear from this example, the notion of prefix and suffix for strings have little to do with the eponymous linguistic terms. 
</div>

In [1]:
def prefixes(string):
    return set(string[:n] for n in range(len(string) + 1))

def suffixes(string):
    return set(string[n:] for n in range(len(string) + 1))

def substrings(string):
    return set(string[m:n] for m in range(len(string)) for n in range(m, len(string)+1))

string = "undo"
print("Substrings of {}: {}".format(string, substrings(string)))
print("Prefixes of {}: {}".format(string, prefixes(string)))
print("Suffixes of {}: {}".format(string, suffixes(string)))

Substrings of undo: {'', 'nd', 'und', 'u', 'd', 'o', 'n', 'undo', 'ndo', 'un', 'do'}
Prefixes of undo: {'', 'und', 'un', 'undo', 'u'}
Suffixes of undo: {'', 'ndo', 'do', 'o', 'undo'}


The definition is very closely related to the one we saw for $n$-grams, which states that $g$ is an $n$-gram of $s$ iff $g \in \Sigma^n$ and there are $u, v \in \Sigma^*$ such that $s = u \tuplecat g \tuplecat v$.
From that perspective, an $n$-gram is just a substring of length $n$.
And in the other direction, the set of substrings for a given string $s$ is just the collection of all $n$-grams such that $n$ does not exceed the length of $s$.
More formally, the set of substrings for $s$ is
$$
    \bigcup_{0 \leq n \leq \length{s}} s_n(s).
$$
The $\bigcup$ operator works similarly to $\sum$ and $\prod$ in that it is a shorthand for multiple operations, in this case multiple unions.
The notation $\length{s}$ indicates the length of the string $s$, and $s_n(s)$ maps every string to its set of $n$-grams.

<div class=example>
We verify that the union-based definition returns the same substrings for *undo*.
$$
\begin{align*}
    \bigcup_{0 \leq n \leq \length{s}} s_n(s)
        =& 
        \bigcup_{0 \leq n \leq 4} s_n(s)
        \\
        =&
        s_0(s) \cup s_1(s) \cup s_2(s) \cup s_3(s) \cup s_4(s)
        \\
        =&
        \setof{\emptystring}
        \cup
        \setof{\text{u}, \text{n}, \text{d}, \text{o}}
        \cup
        \setof{\text{un}, \text{nd}, \text{do}}
        \cup
        \setof{\text{und}, \text{ndo}}
        \cup
        \setof{\text{undo}}
        \\
        =&
        \setof{\emptystring,
        \text{u}, \text{n}, \text{d}, \text{o},
        \text{un}, \text{nd}, \text{do},
        \text{und}, \text{ndo},
        \text{undo}}
        \\
\end{align*}
$$
</div>

In [2]:
def ngrams(string, n):
    return set(string[i:i+n] for i in range(len(string) + 1 - n))

def ngrams_union(string, n):
    return set(ngram for i in range(n + 1) for ngram in ngrams(string, i))

print(substrings(string) == ngrams_union(string, len(string)))

True


So the close connection between substrings and $n$-grams does indeed hold.

## Successor, Precedence, and Subsequences

The notion of substring is also closely tied to the so-called *successor function* $\succ$.
For natural numbers, the successor function returns for each natural number the number right after it: $\succ(n) \is n + 1$.
If we visualize the natural numbers as an infinite string that starts at $0$ and then expands without bounds to the right, each application of the successor function takes us one step farther to the right along this string.
Since strings are a mapping from an initial subset of $\mathbb{N}$ to $\Sigma$, we can also use $\succ$ to take a step to the right through a string.
In *undo*, $\succ(1) = 2$ would amount to taking us from *n* to *d*.
Often it is convenient to use symbols rather than numbers in this case, so that we could write $\succ(\text{n}) = \text{d}$ for the case above.
But this requires that is clear from context which position of the string we have in mind; in *undocumented* $\succ(\text{n})$ could refer to two very different positions with distinct successors.

The connection between substrings and successors is that substrings encode a claim about outputs for the successor function.
Saying that *ndo* is a substring of *undo* means that there is some position $p$ such that

- $p$ is labeled *n*, and
- there is some $q$ such that $\succ(p) = q$ and $q$ is labeled *d*, and
- there is some $r$ such that $\succ(q) = r$ and $r$ is labeled *o*.

The successor function is a very natural concept for strings as it tells us which symbols are adjacent, and $n$ applications of $\succ$ tell us which symbols are separated by $n-1$ symbols.
But sometimes one would instead like to talk about positions that are arbitrarily far away from each other.
In this case the relevant notion is not successor but *precedence*.
Precedence encodes for each position which positions in the string appear to its right. 
Mathematically, it behaves like $<$ for natural numbers, and that's why it is commonly represented by this symbol.

<div class=example>
Consider once more our example string *undo*.
The precedence relations are recorded in the table below, using symbols instead of numbers to indicate positions.
$$
\begin{matrix}
\text{u:} & \text{n} & \text{d} & \text{o}\\
\text{n:} & \text{d} & \text{o}\\
\text{d:} & \text{o}\\
\text{o:} & -\\ 
\end{matrix}
$$
</div>

In [3]:
def pos_symbol(string, pos):
    return (pos, string[pos])

def precedes(string, pos):
    return (pos_symbol(string, pos), [pos_symbol(string, i) for i in range(pos, len(string))])

def precedence_table(string):
    return [precedes(string, pos) for pos in range(len(string))]

def precedence_print(string):
    for row in precedence_table(string):
        pos = row[0][0]
        symbol = row[0][1]
        print("Symbol {} at position {} precedes:".format(symbol, pos))
        for col in row[1]:
            print("\tSymbol {} at position {}".format(col[1], col[0]))
            
string = "undo"
precedence_print(string)

Symbol u at position 0 precedes:
	Symbol u at position 0
	Symbol n at position 1
	Symbol d at position 2
	Symbol o at position 3
Symbol n at position 1 precedes:
	Symbol n at position 1
	Symbol d at position 2
	Symbol o at position 3
Symbol d at position 2 precedes:
	Symbol d at position 2
	Symbol o at position 3
Symbol o at position 3 precedes:
	Symbol o at position 3


In contrast to $\succ$, $\sprec$ is a relation, not a function.
That is because $\sprec$ is not right unique - one symbol may precede many others in a string.
But $\sprec$ is just as useful as $\succ$ for talking about strings.
For instance, look what happens if we take the previous characterization of substring via $\succ$ and replace $\succ$ by $\sprec$.
Then *ndo* is a claim that

- there is some position $p$ labeled $n$, and
- there is some $q$ such that $p \sprec q$ and $q$ is labeled $d$, and
- there is some $r$ such that $q \sprec r$ and $r$ is labeled $o$.

By switching from $\succ$ to $\sprec$, we have turned *ndo* from a substring into a *subsequence*.

<div class=definition>
For every string $s \in \Sigma^*$, $t \is t_1 \tuplecat t_2 \tuplecat \cdots \tuplecat t_n$ is a *subsequence* of $s$ iff there are $u_1, \ldots, u_n, u_{n+1} \in \Sigma^* $ such that $s = u_1 \tuplecat t_1 \tuplecat u_2 \tuplecat t_2 \tuplecat \cdots \tuplecat u_n \tuplecat t_n \tuplecat u_{n+1}$.
</div>

<div class=example>
The string *undo* contains numerous subsequences:
$$
$$\begin{array}{rrrr}
    \text{u} & \text{n} & \text{d} & \text{o}\\
    \text{un} & \text{ud} & \text{uo} & \text{nd} & \text{no} & \text{do}\\ 
    \text{und} & \text{uno} & \text{udo} & \text{ndo}\\
    \text{undo}\\
    \emptystring\\
\end{array}$$
$$
</div>

In [4]:
from itertools import chain, combinations

def k_vectors(string, k):
    return combinations(range(len(string)), k)

def subsequence(string, vector):
    stringlist = list(string)
    return "".join([stringlist[pos] for pos in vector])

def subsequences(string):
    vectors = chain.from_iterable(k_vectors(string, k) for k in range(len(string) + 1))
    return set(subsequence(string, vector) for vector in vectors)

def print_subsequences(string):
    print(sorted(list(subsequences(string)), key=lambda x: len(x)))
    

string = "undo"
print_subsequences(string)

['', 'u', 'd', 'o', 'n', 'nd', 'no', 'ud', 'uo', 'un', 'do', 'uno', 'ndo', 'und', 'udo', 'undo']


Note that every substring is also a subsequence, so the number of substrings can never exceed the number of subsequences.
In general, the number of distinct substrings for a string $s$ is at most
$$
    1 + \sum_{1 \leq n \leq \length{s}} n
$$
whereas the number of distinct subsequences is bounded by $2^{\length{s}}$.
The latter function grows much faster than the former.


## Strictly Piecewise Grammars

So far we have treated $n$-grams as representing substrings of length $n$, but nothing requires us to do so.
We can just as well treat them as representing subsequences of length $n$.
Then we can define a new function for mapping each string to its sets of subsequences of length $n$, and use that to define new variants of local grammars that work with precedence instead of successor.

<div class=definition>
The function $s_n^\sprec: \Sigma^* \rightarrow \wp(\Sigma^n)$ maps each string to its set of subsequences of length $n$.
To clearly differentiate $s_n$ from $s_n^\sprec$, we will henceforth call the former $s_n^\succ$.
</div>

<div class=definition>
A *strictly $k$-piecewise* (SP-$k$) grammar $G$ is a finite set of $k$-grams.
A string $s$ is licensed by $G$ iff $s_k^\sprec(\hat{s}^{k-1}) \cap G = \emptyset$.
</div>

In [5]:
def kgrams_precedence(string, k):
    return set(subsequence(string, vector) for vector in k_vectors(string, k))

def sp_eval(grammar, k, string):
    return kgrams_precedence(string, k).intersection(grammar)

Note how the definition for SP is identical to the one for SL, except that we replaced $s_k^\succ$ by $s_k^\sprec$.

<div class=example>
It is a general property of words that they do not contain more than one primary stress.
Using $'$ as a shorthand for the primary stress marker, we can capture this with the SP-$2$ grammar $\setof{''}$.
Let us calculate the relevant steps for the well-formed *ban'ana* and the ill-formed *'bana'na*.

First $s_2^\sprec(\LeftEdge \text{ba'nana} \RightEdge)$:
$$
\left \{
\begin{matrix}
    \LeftEdge \text{b} & \LeftEdge \text{'} & \LeftEdge \text{a} & \LeftEdge \text{n} & \LeftEdge \RightEdge\\
    \text{ba} & \text{b}' & \text{bn} & \text{b} \RightEdge\\
    \text{a}' & \text{an} & \text{aa} & \text{a} \RightEdge\\
    '\text{n} & '\text{a} & ' \RightEdge\\
    \text{na} & \text{nn} & \text{n} \RightEdge\\
\end{matrix}
\right \}
$$

Intersecting this set with $\setof{''}$ yields the empty set, so the string is well-formed.

Next we calculate $s_2^\sprec(\LeftEdge \text{'bana'na} \RightEdge)$:
$$
\left \{
\begin{matrix}
    \LeftEdge ' & \LeftEdge \text{b} & \LeftEdge \text{a} & \LeftEdge \text{n} & \LeftEdge \RightEdge\\
    '\text{b} & '\text{a} & '\text{n} & '' & ' \RightEdge\\
    \text{ba} & \text{bn} & \text{b}' & \text{b} \RightEdge\\
    \text{an} & \text{aa} & \text{a}' & \text{a} \RightEdge\\
    \text{na} & \text{n}' & \text{nn} & \text{n} \RightEdge\\
\end{matrix}
\right \}
$$
Intersecting this set with the grammar yields $\setof{''}$, which means that the string is ill-formed.
</div>

In [6]:
grammar = {"''"}

for string in ["ba'nana", "'bana'na"]:
    result = sp_eval(grammar, 2, string)
    if result:
        print("{} is ill-formed, contains {}".format(string, ", ".join(result)))
    else:
        print("{} is well-formed".format(string))

ba'nana is well-formed
'bana'na is ill-formed, contains ''


The previous pattern could also be accounted for by a TSL grammar (can you explain how?).
But there are some phonological dependencies in natural language that cannot be captured by TSL grammars but are expressible with SP grammars.

<div class=example>
In some tone languages there is a constraint against low tones (L) occuring within an interval spanned between two high tones (H), no matter how far apart the two high tones are.
So all of the following strings would be illicit: HLH, LHLLLHL, and LLLHLLLLHLLLHLLL.
On the other hand, all of the following are well-formed: HHHH, L, HLLLL, LLLLH, and LLLHHHHLLL.

This split is captured by the SP-$3$ grammar $\setof{\text{HLH}}$.
You are invited to do the calculations yourself.
</div>

In [7]:
grammar = {"HLH"}

for string in ["HLH", "LHLLLHL", "LLLHLLLLHLLLHLLL", "HHHH", "L", "HLLLL", "LLLLH", "LLLHHHHLLL"]:
    result = sp_eval(grammar, 3, string)
    if result:
        print("{} is ill-formed, contains {}".format(string, ", ".join(result)))
    else:
        print("{} is well-formed".format(string))

HLH is ill-formed, contains HLH
LHLLLHL is ill-formed, contains HLH
LLLHLLLLHLLLHLLL is ill-formed, contains HLH
HHHH is well-formed
L is well-formed
HLLLL is well-formed
LLLLH is well-formed
LLLHHHHLLL is well-formed


## Probabilistic Versions

A probabilistic version of SP is easily defined as an analogue of probabilistic SL.
Again we only need to replace $b_n^\succ$ by $b_n^\sprec$, everything else works as before.